In [37]:
import math
import pandas as pd
import numpy as np
from numpy import linalg as la
from scipy import optimize as opt
import matplotlib.pyplot as plt
import scipy

# Calculate bond price / present value
we know that if yield is $r$, then the bond price is 
$$\textbf{price} = \frac{c}{r} + (1-\frac{c}{r})(1 + \frac{r}{k})^{-kN},$$ assuming face value 1.

In [2]:
def get_price(c, k, r, N):
    # assuming face value is 100
    return (c/r + (1-c/r)*(1+r/k)**(-k*N))*100

In [161]:
get_price(0.0263, 2, 0.0237493837204351, 10)

102.25849999999998

In [140]:
def obj_func(r):
    p0 = get_price(0.0525, 2, r, 10)
    return p0 * (1 + r/2)**(25/(181))-120.86

opt.fsolve(obj_func, 0.03)

array([0.02863715])

# Calculate yield


In [157]:
def price_func(r, args):
    # define a function whose input is the yield r
    # the root to the function is the correct yield s.t. the bond is correctly priced
    c, k, N, price = args
    return (c/r + (1-c/r)*(1+r/k)**(-k*N))*100 - price

# we use a solver to solve for the yield that correctly prices each bond
opt.fsolve(price_func, 0.03, args = [0.0263, 2, 10, 102.2585])[0]

0.023749383720435117

# Calculate Macaulay Duration

The formula is, when $k=1,$ given by 
$$\frac{\sum_{k=1}^N k\cdot c_k (1+r)^{-k}}{\sum_{i=1}^{N} c_i (1+r)^{-i}}.$$

In [5]:
# function to calculate MacD for a bond
def get_MacD(N, r, k, c_i):
    # N is the number of years
    # r is yield
    # k is the number of payments per year
    # c_i is an array with k*N elements, payment at each period
    
    # the periods are given by np.linspace(1/k, N, kN)
    
    periods = np.linspace(1/k, N, int(np.round(k*N)))
    # a total of kN payments
    
    numer = 0
    denom = 0

    for i in range(len(periods)):
        numer += periods[i] * c_i[i] * (1+r/k)**(-(i+1))
        denom += c_i[i] * (1+r/k)**(-(i+1))

    return numer/denom


def generate_payments(c, k, N):
    # this function returns an array which is the payments for a bond till maturity
    # c is coupon rate per year
    # k is periods of compounding per year
    # N is years to maturity
    # assuming face value $100
    
    payments = np.ones(int(np.round(k*N)))*(c/k)*100
    payments[-1] += 100
    return payments



In [162]:
get_MacD(10, 0.0237493837204351, 2, generate_payments(0.0263, 2, 10))

8.873602522407797

# Calculate ModD

$$ \textbf{ModD} = \frac{\textbf{MacD}}{1+\frac{r}{k}},$$
where $r$ is annualized yield, $k$ is periods of compounding per year.

$$\left|\textbf{ModD}\cdot \Delta r\right| = \frac{\frac{\partial \textbf{Price}}{\partial r} \Delta r}{\textbf{Price}} = \frac{\Delta \textbf{Price}}{\textbf{Price}}$$
注意方向是反的，yield上升price下降。

In [166]:
def get_ModD(N, r, k, c):
    macD = get_MacD(N, r, k, generate_payments(c, k, N))
    return macD/(1+r/k)


In [171]:
get_ModD(10, 0.0237, 2, 0.0263)

8.76998033012713

In [165]:
def objective_function(a):
    return 1/(a*10**6)-np.exp(-625000*a)

opt.fsolve(objective_function, 100)

/Users/Zeshun/anaconda3/lib/python3.6/site-packages/scipy/optimize/minpack.py:163: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  warnings.warn(msg, RuntimeWarning)


array([1.76023682e+85])

# Calculate optimal portfolio weights

this max the ratio of avg return to stdev

In [9]:
def get_weights(cov_mat, mu, sum_to = 1):
    weights = la.inv(cov_mat)@mu
    tot = np.sum(weights)
    for i in range(len(weights)):
        weights[i] = weights[i]/tot*sum_to
    return weights



In [147]:
C = np.array([[0.043**2, 0.00004], [0.00004, 0.0112**2]])
mu = np.array([0.009, 0.0047])
get_weights(C, mu)

array([0.10149214, 0.89850786])

In [28]:
math.sqrt(0.00012761099458082967)

0.011296503644085177

# Discrete probability, N choose k
scipy.special.comb(100,2)

In [42]:
scipy.special.comb(100,2)

4950.0

In [136]:
t = 0
for i in range(0, 6):
    t += scipy.special.comb(100,i)*(1/38)**i * (37/38)**(100-i)
    
t-1

-0.04900825361731065

# Binary Kelly, $\max \mathbb{E}\ln(R)$

赌1块钱，赢了拿U+1块（净赚U块），输了赔D块钱，默认为1.
initial wealth $W$, what is the optimal percentage of wealth to bet?

$$x = \frac{pU-(1-p)D}{UD}$$


# Bet $x,$ wealth is either $(1+Ux)W$ or $(1-Dx)W$

In [44]:
def binary_kelly(U, D, p):
    numer = p*U-(1-p)*D
    denom = U*D
    return numer / denom


binary_kelly(35, 1, 1/38)

-0.0015037593984962422

In [134]:
def objfunc(x):
    return (1.35**x)*(0.99**(100-x))-2
opt.fsolve(objfunc,2)

array([5.4752661])

# Binary option pricing
price a call option with strike K

stock price S can go up to U, go down to D.

In [81]:
def get_option_price(S, U, D, K, r=0):
    # suppose x shares of stock, y cash
    # the linear system is given by
    # U*x + (1+r)y = U-K
    # D*x + (1+r)y = max{0, D-K}
    mat = np.array([[U, 1+r], [D, 1+r]])
    RHS = np.array([U-K, max(0, D-K)])
    repli_ptf = la.inv(mat)@RHS
    return S*repli_ptf[0] + repli_ptf[1]

def get_replicate_weights(U, D, K, r=0):
    # return x and y, where x is the number of share of stock, y cash position
    mat = np.array([[U, 1+r], [D, 1+r]])
    RHS = np.array([U-K, max(0, D-K)])
    repli_ptf = la.inv(mat)@RHS
    return repli_ptf

def get_replicate_value(stock, up, down, up_pay, down_pay, r = 0):
    mat = np.array([[up, 1+r], [down, 1+r]])
    RHS = np.array([up_pay, down_pay])
    repli_ptf = la.inv(mat)@RHS
    return stock*repli_ptf[0] + repli_ptf[1]

In [65]:
get_option_price(50, 26.4, 7.6, 42)

-35.18297872340426

In [62]:
get_replicate_weights(80, 30, 50)

array([  0.6, -18. ])

In [141]:
get_replicate_value(60, 75, 0, 25, 0)

20.000000000000004

$$\Delta K * \delta = \Delta P$$

# Put-Call parity

$$C(S,t) - P(S,t) = S - K \cdot e^{r(T-t)}$$

# Black Scholes Call option price

$$ C = S_t N(d_1) - K e^{-rt} N(d_2),$$
where $$d_1 = \frac{\ln \frac{S_t}{K} + (r + \frac{\sigma^2}{2})t}{\sigma \sqrt{t}}
, d_2 = d_1 - \sigma \sqrt{t}.$$

K strike price, St current stock price, t time to maturity, r interest rate

delta, 对S求导，N(d1)

theta, 对t求导

vega，对sigma求导

rho，对r求导

gamma，对S的二阶导

In [102]:
from scipy.stats import norm as normal
def bs_call(args):
    St, K, t, r, sigma = args
    d1_numer = np.log(St/K) + (r + 0.5 * sigma**2)*t
    d1_denom = sigma * np.sqrt(t)
    d1 = d1_numer / d1_denom
    d2 = d1 - sigma * np.sqrt(t)
    return St* normal.cdf(d1) - K * np.exp(-r*t)* normal.cdf(d2)


def bs_put(args):
    ## by put call parity
    St, K, t, r, sigma = args
    call_price = bs_call(args)
    return call_price - St + K * np.exp(r*t)

# level payments 等额本息还款

设借了A，每月还X，月利率beta，共m个月，则
$$X = \frac{\beta A}{1 - (1+\beta)^{-m}}$$

In [114]:
def get_monthly_payments(total_borrow, annual_interest_rate, year):
    m = year * 12
    beta = annual_interest_rate / 12
    A = total_borrow
    return (beta * A) / (1 - (1+beta)**(-m))

In [115]:
get_monthly_payments(900000, 0.04, 30)

4296.737659189084

In [172]:
mat = np.array([[104.5, 91.72, 123.86, 102.2],\
                [0.0520, 0.0350, 0.0525, 0.0263],\
               [0.0462, 0.0456, 0.0248, 0.0237],\
               [7.797, 8.253, 8.058, 8.52]])

In [173]:
la.inv(mat)@np.array([100,0.0385, 0.0385, 8])

array([ 5.72452387, -5.10784113, -3.52720518,  3.98393421])